### We are assuming that the state vector are in the following order:
$$ x = [E_{qp}\quad E_{dp}\quad \delta\quad \omega\quad V_F\quad V_A\quad V_E]^T $$

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from rushisland1 import *

In [3]:
who

Aex1	 Aex2	 Bex1	 Bex2	 H	 KA1	 KA2	 KC1	 KC2	 
KD	 KD1	 KD2	 KE1	 KE2	 KF1	 KF2	 Rs	 S2_mat	 
S3_mat	 T2_mat	 T3_mat	 TA1	 TA2	 TE1	 TE2	 TF1	 TF2	 
Tdp	 Tqp	 Trissian	 Xd	 Xdp	 Xl	 Xq	 Xqp	 f_Edp1	 
f_Edp2	 f_Efd	 f_Eqp1	 f_Eqp2	 f_VA1	 f_VA2	 f_VE1	 f_VE2	 f_VF1	 
f_VF2	 f_Vdq	 f_delta1	 f_delta2	 f_w1	 f_w2	 nd	 np	 ratio	 
sys_fun	 


In [7]:
import numpy as np
import numdifftools as nd
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline 
pd.set_option('display.float_format', lambda x: '%.6f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from scipy import optimize
np.set_printoptions(precision=25)

In [9]:
x = np.array([9.8626863024948619e-01,6.1165116423884036e-01,1.2558184342430763e+00,
 -6.2200648344413002e-30,4.8830798594520859e+00,5.4256442882800945e-03,
  2.3355826940514226e+00,9.0728914911947700e-01,6.3456098937805228e-01,
  1.3019464471464004e+00,-3.7087233939572348e-30,2.4973842398326895e+00,
  4.7569223615860759e-02,2.0985781450456096e+00])
# x = np.append(x,[5.769953284494019,5.400027691631498,1.045418310397626,1.077334874177618])

sol = optimize.root(sys_fun, x, method='hybr')
print(sol.fun)
print(sol.message)
print(sol.x)
print(sol.success)
x = sol.x

[-1.6663760219514544e-16 -9.3610710339389268e-16 -4.0979941143246344e-30
  1.0550943450939954e-17  2.5744302020293489e-15  6.1293562817847187e-15
 -2.2773805633336542e-15  0.0000000000000000e+00  0.0000000000000000e+00
  1.6191975338763799e-30  0.0000000000000000e+00  3.7318420995803582e-16
  7.7098821154524763e-18 -2.2204460492503131e-16]
The solution converged.
[ 9.8626863024947953e-01  6.1165116423882404e-01  1.2558184342430696e+00
 -1.0870267000937653e-32  4.8830798594519944e+00  5.4256442882799939e-03
  2.3355826940513795e+00  9.0728914911953285e-01  6.3456098937804051e-01
  1.3019464471463114e+00  4.2950548559772087e-33  2.4973842398325927e+00
  4.7569223615858906e-02  2.0985781450455301e+00]
True


In [6]:
# np.abs(f_Vdq(x))

# Notations

$$ \dot{z} = f(z),\quad f(z_0)=0 $$
$$ \dot{\Delta z} = J\Delta z + \frac{1}{2!}  \Delta z^T H \Delta z + H.O.T(z)$$
$$ \dot{\Delta z} = J\Delta z + \frac{1}{2!}  Z_2*B_z^2 + H.O.T(z)$$
Let $w = Q\Delta z$, and ignore the $\Delta$ sign then
$$Q^{-1}\dot{w} = JQ^{-1} w + \frac{1}{2!} (Q^{-1}w)^T H  (Q^{-1}w) + H.O.T(Q^{-1}w)$$
$$\dot{w} = QJQ^{-1} w + Q \frac{1}{2!} w^T (Q^{-T}H Q^{-1})w + Q * H.O.T(Q^{-1}w)$$ 
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + Q \frac{1}{2!} w^T H_Q w + Q * H.O.T(Q^{-1}w)$$
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + Q \frac{1}{2!}  \begin{bmatrix} w^T H_Q[0] w \\  w^T H_Q[1] w \\ ...  \end{bmatrix} + Q * H.O.T(Q^{-1}w)$$ 
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$$ 
where $W_2$ $W_3$ are constant matrices and $B_w^2$ and $B_w^3$ are column vectors deriving from $w \otimes w$ and $w \otimes w \otimes w$

And we assume that $$w = [u \quad v]^T$$ where $u$ is center variable, $v$ is stable variable

Now assume $B^2_{u}$ is $u\otimes u$ (note that this is a long column vector, so we do not need the $vec$ sign) but ij terms only appear once: e.g. if $u = [u_1 u_2 u_3]^T$, then $B^2_{u}=[u_1^2, u_2u_1, u_3u_1, u_2^2, u_3u_2, u_3^2]$

So we also need two transformation matrices $T$ and $S$ which doing transformations between $B_{uu}$ and $u\otimes u$ like: $$B^2_{u}=T_2 (u\otimes u)$$  
$$u\otimes u=S_2 B^2_{u}$$ 

similarly we will have  
$$B^3_{u}=T_3 (u\otimes B^2_{u})$$  
$$(u\otimes B^2_{u})=S_3 B^3_{u}$$ 

### Now we calculate Jacobian matrix $J$ and  the tranformation matrix $Q$

In [10]:
n_c = 4
n_s = 10
n = x.shape[0]

# Compute Jacobian Matrix J and its eigenvalues
all_fun = [f_Eqp1,f_Edp1,f_delta1,f_w1,f_VF1,f_VA1,f_VE1,f_Eqp2,f_Edp2,f_delta2,f_w2,f_VF2,f_VA2,f_VE2]
J = np.array([nd.Jacobian(f)(x).ravel() for f in all_fun])
#display(pd.DataFrame(J))
lam, v = np.linalg.eig(J.T) # Here we want to calculate the left eigenvecs, so use J_org.T; because use left eigvec makes it easier to calc transformation matrix
print(lam)

[-1.0139228096109090e+01+15.568110691643119j
 -1.0139228096109090e+01-15.568110691643119j
 -9.8673608470149188e-01 +8.201900872809958j
 -9.8673608470149188e-01 -8.201900872809958j
  1.2878587085651816e-14 +5.5363750685276605j
  1.2878587085651816e-14 -5.5363750685276605j
 -7.0192422417378619e+00 +0.j
 -4.9053271494662800e+00 +0.j
  2.4789371944056171e-13 +0.8705493138512748j
  2.4789371944056171e-13 -0.8705493138512748j
 -1.7968170775407084e+00 +0.8122401602367386j
 -1.7968170775407084e+00 -0.8122401602367386j
 -1.7428260325466807e+00 +0.j
 -8.3603203768886136e-01 +0.j                ]


In [14]:
Q = Q = np.block([
    [np.stack((v[:,4].real,v[:,4].imag,v[:,8].real,v[:,8].imag))],
    [v[:,[0,1,2,3,6,7,10,11,12,13]].T]
])
J_cs = np.linalg.multi_dot([Q,J,np.linalg.inv(Q)])
display(pd.DataFrame(J_cs).applymap(lambda x: '{:,.6f}'.format(x)))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.000000+0.000000j,-5.536375-0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000-0.000000j,-0.000000-0.000000j,-0.000000+0.000000j,0.000000-0.000000j,-0.000000+0.000000j,0.000000-0.000000j,0.000000+0.000000j,0.000000+0.000000j,-0.000000-0.000000j
1,5.536375+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000-0.000000j,-0.000000+0.000000j,-0.000000-0.000000j,0.000000-0.000000j,0.000000+0.000000j,-0.000000-0.000000j,-0.000000-0.000000j,-0.000000-0.000000j,-0.000000+0.000000j,0.000000+0.000000j,-0.000000+0.000000j
2,-0.000000-0.000000j,-0.000000+0.000000j,0.000000+0.000000j,-0.870549-0.000000j,-0.000000+0.000000j,-0.000000-0.000000j,-0.000000-0.000000j,-0.000000+0.000000j,-0.000000-0.000000j,-0.000000+0.000000j,-0.000000-0.000000j,-0.000000+0.000000j,0.000000-0.000000j,-0.000000+0.000000j
3,0.000000+0.000000j,0.000000-0.000000j,0.870549+0.000000j,0.000000+0.000000j,0.000000-0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000-0.000000j,-0.000000-0.000000j,0.000000-0.000000j,-0.000000-0.000000j,-0.000000+0.000000j,-0.000000-0.000000j,-0.000000-0.000000j
4,0.000000+0.000000j,-0.000000-0.000000j,0.000000+0.000000j,0.000000-0.000000j,-10.139228+15.568111j,0.000000-0.000000j,0.000000+0.000000j,-0.000000+0.000000j,0.000000-0.000000j,-0.000000-0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,-0.000000+0.000000j
5,0.000000-0.000000j,-0.000000+0.000000j,0.000000-0.000000j,0.000000+0.000000j,0.000000+0.000000j,-10.139228-15.568111j,0.000000-0.000000j,-0.000000-0.000000j,-0.000000+0.000000j,-0.000000+0.000000j,0.000000-0.000000j,-0.000000-0.000000j,0.000000-0.000000j,-0.000000-0.000000j
6,0.000000-0.000000j,-0.000000-0.000000j,0.000000-0.000000j,0.000000-0.000000j,-0.000000-0.000000j,-0.000000-0.000000j,-0.986736+8.201901j,-0.000000+0.000000j,-0.000000+0.000000j,0.000000+0.000000j,-0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,-0.000000+0.000000j
7,0.000000+0.000000j,-0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,-0.000000+0.000000j,-0.000000+0.000000j,-0.000000-0.000000j,-0.986736-8.201901j,-0.000000-0.000000j,-0.000000-0.000000j,0.000000-0.000000j,-0.000000-0.000000j,0.000000-0.000000j,-0.000000-0.000000j
8,-0.000000-0.000000j,0.000000-0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000-0.000000j,0.000000+0.000000j,-0.000000+0.000000j,-0.000000-0.000000j,-7.019242-0.000000j,0.000000-0.000000j,-0.000000-0.000000j,-0.000000+0.000000j,0.000000+0.000000j,-0.000000-0.000000j
9,-0.000000-0.000000j,0.000000-0.000000j,0.000000-0.000000j,0.000000-0.000000j,0.000000-0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000-0.000000j,0.000000-0.000000j,-4.905327+0.000000j,-0.000000-0.000000j,-0.000000-0.000000j,0.000000+0.000000j,-0.000000-0.000000j


In [31]:
# Compute transformation matrix Q
# Q1 = np.block([
#     [np.stack((v[:,4].real,v[:,4].imag,v[:,8].real,v[:,8].imag))],  # Here we know the 4-th and 8-th eigenvalues are what we interested
#     [np.eye(10),np.zeros((10,4))]
# ])
# J1 = np.dot(np.dot(Q1,J),np.linalg.inv(Q1))
# # display(pd.DataFrame(J1))
# Q2 = np.block([
#     [np.eye(4), np.zeros((4,10))],
#     [sp.linalg.solve_sylvester(J1[4:,4:], -J1[0:4,0:4], J1[4:,0:4]), np.eye(10)] #AX+XB=Q
# ]) 
# Q = np.dot(Q2,Q1)
#J_cs = np.dot(np.dot(Q,J_org),np.linalg.inv(Q))

# J_cs = np.linalg.multi_dot([Q,J,np.linalg.inv(Q)])
# display(pd.DataFrame(J_cs))

### Now we can 2nd and 3rd order matrices $W_2$ and $W_3$ 
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$$ 
Since originally we are calculating hessian matrices of $f(z)$, then we need to make transformation from $z$ to $w$, thus
$$ \frac{1}{2} Q Z_2 B_z^2 = \frac{1}{2} Q Z_2[T_2 (z \otimes z)] = \frac{1}{2} Q Z_2 T_2 (Q^{-1} w \otimes Q^{-1} w) = \frac{1}{2} Q Z_2 T_2 (Q^{-1}\otimes Q^{-1})(w\otimes w) = \frac{1}{2} Q Z_2 T_2 (Q^{-1}\otimes Q^{-1})S_2 B_w^2 = W_2 B_w^2$$
Thus
$$\frac{1}{2} Q Z_2 T_2 (Q^{-1}\otimes Q^{-1})S_2 = W_2 $$

For 3rd order
$$\frac{1}{3!} Q Z_3 B_z^3 = \frac{1}{3!} Q Z_3 T_3 (z \otimes B_z^2) = \frac{1}{3!} Q Z_3 T_3 (z \otimes T_2(z \otimes z))$$
where

\begin{align}
&  z \otimes T_2(z \otimes z) \\
& = (I\otimes T_2)(z \otimes z \otimes z) \\
& = (I\otimes T_2)(Q^{-1}w \otimes Q^{-1}w \otimes Q^{-1}w) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(w \otimes w \otimes w) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(w \otimes S_2 B_w^2) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(I \otimes S_2 )(w \otimes B_w^2) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(I \otimes S_2 ) S_3 B_w^3
\end{align}

Thus
$$\frac{1}{3!} Q Z_3 B_z^3 = \frac{1}{3!} Q Z_3 T_3 (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(I \otimes S_2 ) S_3 B_w^3 = W_3 B_w^3$$


In [12]:
Z2 = np.zeros((n,n*(n+1)//2))
for i in range(n):
    hes = nd.Hessian(all_fun[i])(x) # The original Hessian of each f in all_fun
    hes[np.triu_indices(n,1)] *= 2  # double each element above the main diagonal
    Z2[i] = hes[np.triu_indices(n)] # Keep upper triangular part
Z2 = Z2/2 #divide all elements by 2, which corresponds to *2 in above line

In [13]:
# Z3 = np.zeros((n,(n*(n+1)*(n+2)//6)))
# Z3_idx = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)]
# for i in range(n):
#     t = Trissian(all_fun[i], x)
#     Z3[i] = [t[j] for j in Z3_idx]

# Z3_Gain = []
# for i in Z3_idx:
#     val = 1
#     for j in range(n_c):
#         val *= math.factorial(i.count(j))
#     Z3_Gain.append(val)
# Z3_Gain = np.diag(1/np.array(Z3_Gain))    
# Z3 = Z3.dot(Z3_Gain)
# np.savetxt("Fxxx.csv", Z3, delimiter=",")

Z3 = np.genfromtxt('Fxxx_goodcase1.csv', delimiter=',',dtype=float)
display(pd.DataFrame(Z3))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559
0,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.072589,0.000000,0.000000,0.000000,-0.000000,-0.088784,-0.000211,-

In [15]:
W2 = np.linalg.multi_dot([Q, Z2, T2_mat(n), np.kron(np.linalg.inv(Q),np.linalg.inv(Q)), S2_mat(n)])
display(pd.DataFrame(W2).applymap(lambda x: '{:,.6f}'.format(x))) # W2 is of dim: n x n(n+1)/2 = 14 x 105


W3 = np.linalg.multi_dot([Q, Z3, T3_mat(n), np.kron(np.eye(n),T2_mat(n)), 
                      np.kron(np.kron(np.linalg.inv(Q),np.linalg.inv(Q)), np.linalg.inv(Q)), 
                      np.kron(np.eye(n),S2_mat(n)), S3_mat(n)])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104
0,-1.740704-0.000000j,-30.355139+0.000000j,-1.081991-0.000000j,-0.840313-0.000000j,-0.058170+0.132911j,-0.058170-0.132911j,-3.316536+9.204300j,-3.316536-9.204300j,7.586431+0.000000j,-1.529417-0.000000j,1.305934+5.145134j,1.305934-5.145134j,1.938556+0.000000j,0.020335+0.000000j,-73.617785+0.000000j,5.790618+0.000000j,0.597047+0.000000j,-0.106235+1.352961j,-0.106235-1.352961j,0.304864-0.311485j,0.304864+0.311485j,13.641912-0.000000j,37.769749-0.000000j,2.268697+19.732708j,2.268697-19.732708j,14.271308-0.000000j,-0.096669-0.000000j,-0.577715-0.000000j,-0.521082-0.000000j,-0.047508+0.020253j,-0.047508-0.020253j,-1.662223+4.370853j,-1.662223-4.370853j,2.986655-0.000000j,-2.518695+0.000000j,0.315753+0.458119j,0.315753-0.458119j,-0.636051-0.000000j,0.021034+0.000000j,-0.118622-0.000000j,-0.017083+0.043852j,-0.017083-0.043852j,-1.189247+2.245410j,-1.189247-2.245410j,2.304586-0.000000j,-1.334476+0.000000j,0.138507+0.509962j,0.138507-0.509962j,-0.077429-0.000000j,0.008835+0.000000j,-0.011110-0.060314j,-0.122358+0.000000j,0.293762+0.193493j,-0.302405-0.251001j,-0.121849-0.139439j,-0.132329-0.290029j,0.186354-0.087299j,-0.241897-0.170114j,-0.000540-0.055355j,0.000604-0.001083j,-0.011110+0.060314j,-0.302405+0.251001j,0.293762-0.193493j,-0.121849+0.139439j,-0.132329+0.290029j,-0.241897+0.170114j,0.186354+0.087299j,-0.000540+0.055355j,0.000604+0.001083j,10.254232+7.794089j,-17.347271+0.000000j,-1.868917-16.169469j,3.631175+6.345192j,6.751864+0.332083j,-3.418361-3.726842j,-0.951472+0.667123j,0.041256-0.080068j,10.254232-7.794089j,-1.868917+16.169469j,3.631175-6.345192j,-3.418361+3.726842j,6.751864-0.332083j,-0.951472-0.667123j,0.041256+0.080068j,0.262208+0.000000j,-7.337237+0.000000j,-3.482163-5.663048j,-3.482163+5.663048j,0.662761-0.000000j,-0.074940+0.000000j,1.359527+0.000000j,1.193445-2.064112j,1.193445+2.064112j,-4.013841+0.000000j,0.059160+0.000000j,1.827129-0.763336j,-3.734419+0.000000j,-0.287591-1.546587j,-0.004051-0.009453j,1.827129+0.763336j,-0.287591+1.546587j,-0.004051+0.009453j,-0.574254-0.000000j,0.009313+0.000000j,-0.000180-0.000000j
1,-0.019981-0.000000j,-4.244885-0.000000j,1.042357-0.000000j,0.627526-0.000000j,-0.101568-0.430704j,-0.101568+0.430704j,-1.464751+7.165191j,-1.464751-7.165191j,-0.858935-0.000000j,-1.938267+0.000000j,-0.961608-0.854368j,-0.961608+0.854368j,0.724735+0.000000j,-0.024382+0.000000j,-56.407260+0.000000j,1.507892-0.000000j,0.140015-0.000000j,-0.518248-1.314698j,-0.518248+1.314698j,-1.060533+12.103656j,-1.060533-12.103656j,-2.202172-0.000000j,2.739063+0.000000j,0.112946+6.016017j,0.112946-6.016017j,4.194466+0.000000j,-0.026515+0.000000j,0.523634+0.000000j,0.699532-0.000000j,0.054812-0.130257j,0.054812+0.130257j,-1.054505+0.777605j,-1.054505-0.777605j,-0.096851-0.000000j,-1.804355+0.000000j,-1.012700-1.498395j,-1.012700+1.498395j,0.383232-0.000000j,-0.024534+0.000000j,0.238824-0.000000j,0.002959-0.140457j,0.002959+0.140457j,-0.736771+0.734778j,-0.736771-0.734778j,-0.235509-0.000000j,-1.311357+0.000000j,-0.744022-1.080296j,-0.744022+1.080296j,0.221018+0.000000j,-0.016607+0.000000j,0.013274+0.121227j,0.265439-0.000000j,-0.973224+0.037778j,0.818334+0.072901j,0.412992+0.401387j,0.311223+0.662965j,-0.408814+0.366202j,0.592494+0.285667j,0.025691+0.062224j,-0.000333+0.004104j,0.013274-0.121227j,0.818334-0.072901j,-0.973224-0.037778j,0.412992-0.401387j,0.311223-0.662965j,0.592494-0.285667j,-0.408814-0.366202j,0.025691-0.062224j,-0.000333-0.004104j,17.304972+2.512578j,-32.963659+0.000000j,-0.051531+2.203340j,1.724666-0.371970j,2.713769+1.118337j,-0.520663-2.430678j,-0.235528+0.136223j,0.024817-0.020223j,17.304972-2.512578j,-0.051531-2.203340j,1.724666+0.371970j,-0.520663+2.430678j,2.713769-1.118337j,-0.235528-

### Now we are able to calculate center manifold up to 2nd order terms, which we denote it by:
$$v = h_{c2}(u) = \color{red}{H_{c2}}B_u^2$$
where $\color{red}{H_{c2}} \in \mathbb{R}^{n_s \times n_c(n_c+1)/2}$ is the matrix we want to find; the subscript $c$ means center manifold (to distinguish it from normal form later), and 2 means 2nd order

Then according to equation:
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$$ 
if we express the term $W_2 B_w^2$ in form of $u$ and $v$, we will have
$$ \dot{u} = J_c u + f_2(u,v) = J_c u + \color{red}{U_u^2} B_u^2 + \color{red}{U_{uv}} B_{uv} + U_v^2 B_v^2 + ... $$
$$ \dot{v} = J_s v + g_2(u,v) = J_s v + \color{red}{V_u^2} B_u^2 + V_{uv} B_{uv} + V_v^2 B_v^2 + ... $$

Since we can find $\color{red}{H_{c2}}$ by solving the sylvester equation: $\color{red}{J_s H_{c2} + H_{c2} C_2 = -V_u^2}$ so we need to calculate the matrix of $V_u^2$

where $C_2$ comes from
$$ (DB_u^2)J_c u = T_2 D(u \otimes u) J_c u = T_2 \bar{J_c}(u \otimes u) = T_2 \bar{J_c} S_2 B_u^2 = -C_2 B_u^2$$
and
$$\bar{J_c} = (I_n \otimes J_c) + \begin{pmatrix} I_n \otimes J_1\\I_n \otimes J_2 \\ ... \\I_n \otimes J_n \end{pmatrix}$$ where each $J_i$ is the i-th row of matrix $J_c$

Now we construct matrix <font color = 'red'>$V_u^2$, $\bar{J_c}$ $C_2$ $H_{c2}$ </font>

In [16]:
J_c = J_cs[0:n_c,0:n_c]
J_s = J_cs[n_c:,n_c:]
V2_uu = np.zeros((n_s,n_c*(n_c+1)//2))

W2_idx = [(i,j) for i in range(n) for j in range(i,n)]
V2_uu_idx = [(i,j) for i in range(n_c) for j in range(i,n_c)]
V2_uu_idx = [W2_idx.index(i) for i in V2_uu_idx]
V2_uu = np.array([w[V2_uu_idx] for w in W2[n_c:n]])
# display(pd.DataFrame(V2_uu))

J_cbar = np.kron(np.eye(n_c),J_c) + np.array([np.kron(np.eye(n_c),row) for row in J_c]).reshape(-1,n_c**2)
#display(pd.DataFrame(J_cbar))

C2 = - np.linalg.multi_dot([T2_mat(n_c),J_cbar,S2_mat(n_c)])
#display(pd.DataFrame(C2))

H_c2 = sp.linalg.solve_sylvester(J_s, C2, -V2_uu)
# display(pd.DataFrame(H_c2))

### Now we can plug $v = {H_{c2}}B_u^2$ into equation of $\dot{u}$, then we can get up to the 3rd order terms
Since we have 
$$ \dot{u} = J_c u + f_2(u,v) + f_3(u,v) + ... $$
by plugging 
$$v = h_{c2}(u) = H_{c2} B_u^2$$
into the above equation, for third order terms of $u$, we only need:
$$ \dot{u} = J_c u + f_2(u,H_{c2} B_u^2) + f_3(u,0) + ... $$
Let
$$f_2(u,v) = U_u^{2} B_u^2 + U_{uv} B_{uv} + U_v^{2} B_v^2$$
and 
$$f_3(u,0) = U_u^{3} B_u^3$$
Then we can see that $B_{uv}$ is in 3rd order of $u$. $B_v^2$ is in 4th order of $u$, which is ignored here.
So the center manifold up to 3rd order of $u$ could be expressed as:
$$\dot{u} = J_c u + U_u^{2} B_u^2 + U_{uv} B_{uv} + U_u^{3} B_u^3 + O(u^4) $$
Since 
$$B_{uv} = u \otimes v = u \otimes H_{c2} B_u^2 = (I_{n_c} \otimes H_{c2})(u \otimes B_u^2) = (I_{n_c} \otimes H_{c2})S_3 B_u^3$$
Thus
$$\dot{u} = J_c u + U_u^{2} B_u^2 + U_{uv} (I_{n_c} \otimes H_{c2})S_3 B_u^3 + U_u^{3} B_u^3 + O(u^4) = J_c u + \color{red}{U_u^{2}} B_u^2 + [\color{red}{U_{uv}} (I_{n_c} \otimes H_{c2})\color{red}{S_3} + \color{red}{U_u^{3}}] B_u^3 + O(u^4)$$

So here in order to get the know the equation of $\dot{u}$ up to 3rd order, we need to know the following matrices: <font color = 'red'>$U_u^{2}$, $U_{uv}$, $S_3$, $U_u^{3}$</font>.

Here in order to know $U_u^{3}$, which comes from $f_3(u,0)$, we need to calculate matrix $W_3$ in equation $\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$

### Firstly we calculate $U_u^2$ and $U_{uv}$. Both of them comes from $W_2$

In [17]:
Uu2_idx = [(i,j) for i in range(n_c) for j in range(i,n_c)]
Uu2_idx = [W2_idx.index(i) for i in Uu2_idx]
Uu2 = np.array([w[Uu2_idx] for w in W2[0:n_c]])
#display(pd.DataFrame(Uu2))
Uuv_idx = [(i,j) for i in range(n_c) for j in range(n_c,n)]
Uuv_idx = [W2_idx.index(i) for i in Uuv_idx]
Uuv = np.array([w[Uuv_idx] for w in W2[0:n_c]])
#display(pd.DataFrame(Uuv))

### Now we are ready to calculate matrix $U_u^3$. 
But in order to calculate it, we need to calculate the 3rd order derivative matrix $W_3$, then formulize $U_u^3$

Now we have known the following matrices: <font color = 'red'>$U_u^{2}$, $U_{uv}$, $S_3$  and $U_u^{3}$ </font>

Then we can calculate 
$$U_{uc}^3 = U_{uv} (I_{n_c} \otimes H_{c2})S_3 + U_u^{3}$$ 
in the center manifold:
$$\dot{u} =  J_c u + U_u^{2} B_u^2 + [U_{uv} (I_{n_c} \otimes H_{c2})S_3 + U_u^{3}] B_u^3 + O(u^4) = J_c u + U_u^{2} B_u^2 + U_{uc}^3 B_u^3 + O(u^4)$$

In [18]:
W3_idx = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)]
Uu3_idx = [(i,j,k) for i in range(n_c) for j in range(i,n_c) for k in range(j,n_c)]
Uu3_idx = [W3_idx.index(i) for i in Uu3_idx]
Uu3 = np.array([w[Uu3_idx] for w in W3[0:n_c]])
#display(pd.DataFrame(Uu3))
Uuc3 = np.linalg.multi_dot([Uuv, np.kron(np.eye(n_c), H_c2), S3_mat(n_c)]) + Uu3

### <font color='red'>Now we've got the flow $\dot{u}$ on the center manifold up to 3rd order, the next step is to calculate the normal form</font>

<font color='red'>$$\dot{u} = J_c u + U_u^{2} B_u^2 + U_{uc}^3 B_u^3 + O(u^4)$$
至此，我们只需要研究这个方程，我们已经知道$J_c$, $U_u^{2}$, $U_{uc}$,那么接下来就是要算normalform
</font>
#### First, we calculate the normal form up to the 2nd order:
$$u = y + \color{red}{H_{2n}} B_y^2$$
where we want to find matrix $\color{red}{H_{2n}}\in \mathbb{R}^{n_c \times n_c(n_c+1)/2}$

Then by plugging it into the above equation, we will get:
$$(I + H_{2n} DB_y^2) \dot{y} = J_c (y + H_{2n} B_y^2) + U_u^2 B_u^2 + U_{uc}^3 B_u^3 + O(u^4)$$
<font color='red'>$$\dot{y} = (I + H_{2n} DB_y^2)^{-1} [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_u^2 + U_{uc}^3 B_u^3 + O(u^4)]$$</font>
Since by some derivation, we know:
$$B_u^2 = B_y^2 + T_2\bar{H}(I\otimes S_2)S_3 B_y^3$$
where
$$\bar{H} = (I_n \otimes H) + \begin{pmatrix} I_n\otimes h_1 \\I_n\otimes h_2 \\ \vdots \\ I_n\otimes h_n \\ \end{pmatrix}$$
$H = H_{2n}T_2$ ($T_2=T_2(n_c)$) and $h_1$ ... $h_n$ are rows of matrix $H$.
In the above equations, if we assume $u\in \mathbb{R}^{n_c}$, then $H_{2n} \in \mathbb{R}^{n_c \times \frac{n_c(n_c+1)}{2}}$, $H \in \mathbb{R}^{n_c\times n_c^2}$, $\bar{H} \in \mathbb{R}^{n_c^2 \times n_c^3}$, $S_2=S_2(n_c)$, $S_3=S_3(n_c)$

Then
<font color='red'>$$\dot{y} = (I - H_{2n} DB_y^2)  [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_y^2 + U_u^2 T_2\bar{H}(I\otimes S_2)S_3 B_y^3 + U_{uc}^3 B_u^3 + O(u^4)]$$</font>
**Then up to 2nd order terms on the right hand side is**:
$$\dot{y} = J_c y + J_c H_{2n} B_y^2 + U_u^2 B_y^2 - H_{2n} DB_y^2 J_c y + O(y^3)$$
If we denote $- DB_y^2 J_c y = \color{red}{C_2} B_y^2$ as we did in calculation of center manifold, then:
$$\dot{y} = J_c y + [J_c H_{2n} + U_u^2 + H_{2n} C_2] B_y^2 + O(y^3)$$
Due to resonance terms, $J_c H_{2n} + H_{2n} C_2 + U_u^2 $ might not be totally eliminated, so we let
<font color='red'>$$J_c H_{2n} + H_{2n} C_2 = - U_u^2 + R_{2n} $$</font>
where matrix $R_{2n}$ represents resonance terms in normal form.

Now the problem is trying to solve for $H_{2n}$ which makes $R_{2n}$ as simple as possible, where $J_c$, $U_{u}^2$ and $C_2$ are known

Since $J_c \in \mathbb{R}^{n_c\times n_c}$ , $H_{2n} \in \mathbb{R}^{n_c\times \frac{n_c(n_c+1)}{2}}$  and $C_2 \in \mathbb{R}^{\frac{n_c(n_c+1)}{2} \times \frac{n_c(n_c+1)}{2}}$
First we could transform the equation into:
$$\underbrace{(I_{\frac{n_c(n_c+1)}{2}}\otimes J_c + C_2^T \otimes I_{n_c})}_\text{$\Lambda_2$} \cdot \text{vec} (H_{2n}) = -\text{vec}(U_u^2) + \text{vec}(R_{2n})$$


Since we know that for Hopf Bifurcation, the 2nd order terms in normal form will all be eliminated, so $\Lambda$ should be a full rank matrix and the resonance term $R_{2n}$ will not exist, which is also verified below. Then we could solve for $H_{2n}$ by directly solving the Sylvester equation $J_c H_{2n} + H_{2n} C_2 = - U_u^2$

In [19]:
Lambda2 = np.kron(np.eye(n_c*(n_c+1)//2),J_c) + np.kron(C2.T, np.eye(n_c))
# Since we know 2nd order will be eliminated, so Lambda should be full rank
assert(np.linalg.matrix_rank(Lambda2) == Lambda2.shape[0])
# So R_2n will be zero and H_2n could be solved directly from the Sylvester Equation
H_2n = sp.linalg.solve_sylvester(J_c, C2, -Uu2)
#display(pd.DataFrame(H_2n))
R_2n = np.dot(J_c,H_2n) + np.dot(H_2n,C2) + Uu2

In [20]:
# dummy1, dummy2 = np.linalg.eig(Lambda2)
# print(dummy1)
display(pd.DataFrame(R_2n).applymap(lambda x: '{:,.6f}'.format(x)))

,0,1,2,3,4,5,6,7,8,9
0,0.000000+0.000000j,0.000000+0.000000j,-0.000000+0.000000j,0.000000+0.000000j,0.000000-0.000000j,-0.000000+0.000000j,-0.000000+0.000000j,0.000000-0.000000j,0.000000+0.000000j,0.000000-0.000000j
1,0.000000-0.000000j,-0.000000-0.000000j,-0.000000-0.000000j,0.000000-0.000000j,-0.000000-0.000000j,0.000000-0.000000j,0.000000+0.000000j,-0.000000-0.000000j,-0.000000+0.000000j,0.000000-0.000000j
2,0.000000+0.000000j,-0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,-0.000000-0.000000j,0.000000+0.000000j,-0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,-0.000000-0.000000j
3,-0.000000-0.000000j,-0.000000+0.000000j,0.000000+0.000000j,-0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,-0.000000+0.000000j,-0.000000-0.000000j,-0.000000-0.000000j,0.000000-0.000000j


### Next, we calculate the normal form up to the 3rd order terms
From previous equation:
$$\dot{y} = (I + H_{2n} DB_y^2)^{-1} [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_u^2 + U_{uc}^3 B_u^3 + O(u^4)]$$
<font color='red'>$$\dot{y} = (I - H_{2n} DB_y^2)  [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_y^2 + U_u^2 T_2\bar{H}(I\otimes S_2)S_3 B_y^3 + U_{uc}^3 B_u^3 + O(u^4)]$$</font>
Since if we only consider 3rd order terms, $U_{uc}^3 B_u^3$ can be expressed as:
$$ U_{uc}^3 B_u^3 = U_{uc}^3 B_y^3$$
So 
$$\dot{y} = [I - \underbrace{H_{2n} DB_y^2}_\text{O(y)} + \underbrace{(H_{2n} DB_y^2)(H_{2n} DB_y^2)}_\text{O(y^2)}+O(y^3)] [J_c y + \underbrace{(J_c H_{2n} + U_u^2) B_y^2}_\text{O(y^2)} + \underbrace{[U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3}_\text{O(y^3)} + O(u^4)]$$
Thus the 3rd order terms on the right hand side is:

\begin{align}
& [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} DB_y^2 (J_c H_{2n} + U_u^2) B_y^2 + (H_{2n} DB_y^2)(H_{2n} DB_y^2)J_c y \\
= & [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} DB_y^2 [(J_c H_{2n} + U_u^2) B_y^2 - H_{2n} DB_y^2 J_c y] \\
= & [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} DB_y^2 [(J_c H_{2n} + U_u^2) B_y^2 + H_{2n} C_2 B_y^2] \\
= & [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} \underbrace{DB_y^2 R_{2n} B_y^2}_\text{O(y^3)} 
\end{align}

where the sizes of matrices are:
\begin{align}
& H_{2n}, U_{u2} --  n_c \times \frac{n_c(n_c+1)}{2}  \\
& T_2(n_c) -- \frac{n_c(n_c+1)}{2} \times n_c^2  \\
& \bar{H}, \bar{R} -- n_c^2 \times n_c^3  \\
& I\otimes S_2(n_c) -- (n_c \times n_c) \otimes (n_c^2 \times \frac{n_c(n_c+1)}{2})  \\
& S_3(n_c) -- \frac{n_c^2 n_c(n_c+1)}{2} \times \frac{n_c(n_c+1)(n_c+2)}{3!}  \\
& U_{uc}^3 -- n_c \times \frac{n_c(n_c+1)(n_c+2)}{3!}  \\
\end{align}

So we only need to calculate 
$$ DB_y^2 R_{2n} B_y^2 = T_2 \bar{R} (I \otimes S_2) S_3 B_y^3$$
where $$\bar{R} = (I_n\otimes R) + \begin{pmatrix} I_n\otimes r_1 \\ I_n\otimes r_2 \\ \vdots \\ I_n\otimes r_n \end{pmatrix} $$
where $R = R_{2n}T_2$ and $r_i$ are rows of $R$

Thus the 3rd order terms are:
$$N_y^3 B_y^3 = [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3 - H_{2n} T_2 \bar{R} (I \otimes S_2) S_3 ]B_y^3$$
where
$$\bar{H} = (I_n \otimes H) + \begin{pmatrix} I_n\otimes h_1 \\I_n\otimes h_2 \\ \vdots \\ I_n\otimes h_n \\ \end{pmatrix}$$
$H = H_{2n}T_2$ and $h_1$ ... $h_n$ are rows of matrix $H$

Now the normal form is:
$$\dot{y} = J_c y + R_{2n}B_y^2 + N_y^3 B_y^3 + O(y^4)$$

Now we want to simply $N_y^3$ as many as possible, so we consider another 3rd order nonlinear transformation 
$$ y = z + H_{3n}B_z^3$$

Here if we assume $y,z \in \mathbb{R}^{n_c}$ then $H_{3n} \in \mathbb{R}^{n_c \times \frac{n_c(n_c+1)(n_c+2)}{3!}}$

Similarly, we will have 
$$\dot{z} = J_c z + R_{2n}B_z^2 + R_{3n} B_z^3 + O(z^4)$$
where
$$R_{3n} = J_c H_{3n} + H_{3n}C_3 + N_y^3$$
where 
$$C_3 B_z^3 = -DB_z^3 J_c z$$ and $$C_3 = -T_3(I_n\otimes T_2)\bar{J}(I_n\otimes S_2)S_3$$
where
$$\bar{J} = (I_{n^2} \otimes J_c) + I_n \otimes \begin{pmatrix} I_n\otimes jc_1 \\I_n\otimes jc_2 \\ \vdots \\ I_n\otimes jc_n \\ \end{pmatrix} + \begin{pmatrix} I_{n^2}\otimes jc_1 \\I_{n^2}\otimes jc_2 \\ \vdots \\ I_{n^2}\otimes jc_n \\ \end{pmatrix}$$

$jc_i$ are rows of matrix $J_c$

So here we have $R_{3n} \in \mathbb{R}^{n_c \times \frac{n_c(n_c+1)(n_c+2)}{3!}}$, $C_3 \in \mathbb{R}^{\frac{n_c(n_c+1)(n_c+2)}{3!} \times \frac{n_c(n_c+1)(n_c+2)}{3!}}$, 

So in order to calculate $R_{3n}$, first we need to calculate $N_y^3$.
Let's calcuate $\bar{H}$ and $\bar{R}$ first

In [21]:
H_bar = np.kron(np.eye(n_c),np.dot(H_2n,T2_mat(n_c))) + np.array([np.kron(np.eye(n_c),row) for row in np.dot(H_2n,T2_mat(n_c))]).reshape(-1,n_c**3)
#display(pd.DataFrame(H_bar))
R_bar = np.kron(np.eye(n_c),np.dot(R_2n,T2_mat(n_c))) + np.array([np.kron(np.eye(n_c),row) for row in np.dot(R_2n,T2_mat(n_c))]).reshape(-1,n_c**3)
#display(pd.DataFrame(R_bar))
Ny3 = np.linalg.multi_dot([Uu2,T2_mat(n_c),H_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)]) + Uuc3 - np.linalg.multi_dot([H_2n,T2_mat(n_c),R_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)])
#display(pd.DataFrame(Ny3))
J_bar = np.kron(np.eye(n_c**2),J_c) + np.kron(np.eye(n_c), np.array([np.kron(np.eye(n_c),row) for row in J_c]).reshape(-1,n_c**2)) + np.array([np.kron(np.eye(n_c**2),row) for row in J_c]).reshape(-1,n_c**3) 
#display(pd.DataFrame(J_bar))
C3 = -np.linalg.multi_dot([T3_mat(n_c),np.kron(np.eye(n_c),T2_mat(n_c)),J_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)])
#display(pd.DataFrame(C3))

Then we tranform the equation:
$$R_{3n} = J_c H_{3n} + H_{3n}C_3 + N_y^3$$
into:
$$(I_{\frac{n_c(n_c+1)(n_c+2)}{3!}} \otimes J_c + C_3^T \otimes I_{n_c}) * \text{vec}(H_{3n}) = \text{vec}(R_{3n}) - \text{vec}(N_y^3)$$
Let
$$\Lambda_3 = I_{\frac{n_c(n_c+1)(n_c+2)}{3!}} \otimes J_c + C_3^T \otimes I_{n_c}$$
<font color='red'>
Here we are assuming the $vec$ operation is taking the column vectors of the matrix. However, it is more easily for us to interprete the results if we assume that $vec$ is taking the row vectors of the matrix. But here the $\Lambda_3$ would be changing to
    $$ \Lambda_3 = J_c \otimes I_{\frac{n_c(n_c+1)(n_c+2)}{3!}} + I_{n_c} \otimes C_3^T$$
</font>

In [23]:
Lambda3 = np.kron(J_c,np.eye(n_c*(n_c+1)*(n_c+2)//6)) + np.kron(np.eye(n_c),C3.T)
lam, _ = np.linalg.eig(Lambda3)
lam

array([-2.1316282072803006e-14-2.2145500274110621e+01j,
       -2.3092638912203256e-14+2.2145500274110603e+01j,
       -2.1316282072803006e-14-1.1072750137055337e+01j,
       -3.2098474895978163e-14+1.1072750137055358e+01j,
        2.0819338007044401e-13-1.7479674519434276e+01j,
        2.0383822125873141e-13+1.7479674519434202e+01j,
       -2.5507373990762972e-13-1.7479674519434326e+01j,
       -2.6822988274943782e-13+1.7479674519434234e+01j,
        2.1546653348973053e-13-1.5738575891731760e+01j,
       -2.4920733879274515e-13-1.5738575891731751e+01j,
        2.1671553620428837e-13+1.5738575891731717e+01j,
       -2.7502999877526690e-13+1.5738575891731678e+01j,
       -2.5769240465063895e-14-1.1072750137055341e+01j,
       -1.6170237076810775e-14+1.1072750137055337e+01j,
       -2.2842838731662596e-14-1.2813848764757861e+01j,
       -4.9565429885434220e-13-1.2813848764757859e+01j,
       -1.9040324872321356e-14+1.2813848764757882e+01j,
       -4.9484179774628156e-13+1.281384876475787

In [24]:
Lambda3.shape

(80, 80)

Now Since we know that the column vector $-vec(N_y^3)$ is in the column space of matrix $\Lambda_3$ but vector $vec(R_3^n)$ is not. This means vector $vec(R_3^n)$ is in the complement space of the column space of matrix $\Lambda_3$. Since we have already know what the normal form with 3rd order resonance terms looks like (shown below), so we could easiliy know the basis of the space where vector $vec(R_3^n)$ is in.

$$ \dot{z}=i\omega_1z+a_1z^2\bar{z}+a_2zw\bar{w}+\ldots $$ 
$$ \dot{w}=i\omega_2w+b_1w^2\bar{w}+b_2z\bar{z}w+\ldots $$


$$ \dot{x_1}=-\omega_1x_2+(a_1^Rx_1-a_1^Ix_2)(x_1^2+x_2^2)+(a_2^Rx_1-a_2^Ix_2)(x_3^2+x_4^2)$$
$$ \dot{x_2}=\omega_1x_1+(a_1^Rx_2+a_1^Ix_1)(x_1^2+x_2^2)+(a_2^Rx_2+a_2^Ix_1)(x_3^2+x_4^2) $$
$$ \dot{x_3}=-\omega_2x_4+(b_1^Rx_3-b_1^Ix_4)(x_3^2+x_4^2)+(b_2^Rx_3-b_2^Ix_4)(x_1^2+x_2^2)$$ 
$$ \dot{x_4}=\omega_2x_3+(b_1^Rx_4+b_1^Ix_3)(x_3^2+x_4^2)(b_2^Rx_4+b_2^Ix_3)(x_1^2+x_2^2)$$

In [25]:
L3 = np.zeros((80,8))
L3[[0,4,21,30],[0,0,0,0]] = 1 # a1_R
L3[[7,9,33,35],[2,2,2,2]] = 1 # a2_R
L3[[56,58,77,79],[4,4,4,4]] = 1 # b1_R
L3[[42,51,63,72],[6,6,6,6]] = 1 # b2_R

L3[[1,10],[1,1]] = -1
L3[[20,24],[1,1]] = 1   #a1_I
L3[[13,15],[3,3]] = -1
L3[[27,29],[3,3]] = 1    #a2_I
L3[[57,59],[5,5]] = -1
L3[[76,78],[5,5]] = 1    #b1_I
L3[[43,52],[7,7]] = -1
L3[[62,71],[7,7]] = 1    #b2_I

# np.linalg.matrix_rank(L3)
print(np.linalg.matrix_rank((np.concatenate((Lambda3,L3),axis=1))))
assert(np.linalg.matrix_rank((np.concatenate((Lambda3,L3),axis=1))) == Lambda3.shape[0])

80


In [20]:
# n_c = 4
# for i,v in enumerate([(i,j,k) for i in range(1,n_c+1) for j in range(i,n_c+1) for k in range(j,n_c+1)]):
#     print(i,v)

In [30]:
P, L, U = sp.linalg.lu(Lambda3,permute_l=False)
L = P.dot(L)
L_inv = np.linalg.inv(L)

U_zero_rows = np.where(abs(np.diag(U)) < 1e-10)[0]
L2 = L_inv[U_zero_rows,:]

In [31]:
U_zero_rows

array([24, 29, 30, 35, 71, 72, 78, 79], dtype=int64)

In [32]:
# pd.DataFrame(U[U_zero_rows,:].T)
np.where(abs(U[U_zero_rows,:]) > 1e-10)

(array([], dtype=int64), array([], dtype=int64))

In [33]:
theta =np.linalg.inv(L2.dot(L3)).dot(L2).dot(Ny3.reshape((1,-1)).T)
theta

array([[-6.3470132632867512e+02-1.2527680160163603e-12j],
       [-2.6147414977421499e+03+5.0862021945694673e-13j],
       [ 5.3776054424438664e+00+1.4077552664750164e-14j],
       [-4.4578495401450326e+01+2.8432371460318382e-14j],
       [-2.6242139730726772e+00-9.4458526415742561e-15j],
       [ 2.3398918331659502e-01-1.7119223927119381e-15j],
       [-7.1408486548228325e+00+2.6667956629659905e-13j],
       [ 7.7413993165652641e+02-3.2565457347462629e-13j]])

In [23]:
# array([[-6.3470132634001288e+02],
#        [-2.6147414977491276e+03],
#        [ 5.3776054423841293e+00],
#        [-4.4578495401119525e+01],
#        [-2.6242139731366096e+00],
#        [ 2.3398918316453976e-01],
#        [-7.1408486518023437e+00],
#        [ 7.7413993164544945e+02]])

In [59]:
# fig,ax = plt.subplots(figsize=(10,10))
# ax.spy(U, precision=0.1, markersize=5)
# plt.grid(True)

$$ \dot{r_1}=\mu_1 r_1+a_1^R r_1^3+a_2^R r_1 r_2^2+\ldots $$ 
$$ \dot{r_2}=\mu_2 r_2+b_1^R r_2^3+b_2^R r_1^2 r_2+\ldots $$ 

In [34]:
# a1_R, a2_R, b1_R, b2_R
ab_R = theta[0::2]
ab_R

array([[-634.7013263286751   -1.2527680160163603e-12j],
       [   5.377605442443866 +1.4077552664750164e-14j],
       [  -2.6242139730726772-9.4458526415742561e-15j],
       [  -7.1408486548228325+2.6667956629659905e-13j]])

In [28]:
5.377605442384129/-2.6242139731366096

-2.049225214648373

In [29]:
-7.140848651802344/-634.7013263400129

0.01125072275014745